Lets Begin Coding
1. Download and Install required modules and libraries
2. Import required data and connect with drive



Importing all files and models

In [ ]:

!pip install lpips
!pip install torch torchvision
#!pip install mxnet-cu90 #optional, for data processing MXNet 1.3.1
!pip install focal-loss-torch
!pip install split-folders # Used for split folders into train and val datasets or in our case split data into gallery and probe
!pip install PTable
#https://torchmetrics.readthedocs.io/en/latest/image/learned_perceptual_image_patch_similarity.html

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 KB 1.9 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for PTable: filename=PTable-0.9.2-py3-none-any.whl size=22925 sha256=a9ed4e2fed75321b4f615b2802594b612d43402be2b89e6ddd5a937486b22f8c
  Stored in directory: /root/.cache/pip/wheels/1b/3a/02/8d8da2bca2223dda2f827949c88b2d82dc85dccbc2bb6265e5
Successfully built PTable


In [ ]:
import torch
from PIL import Image
import numpy as np
import lpips
import torchvision.transforms as transforms
import h5py
import matplotlib.pyplot as plt
%matplotlib inline
# used to supress display of warnings
import warnings
from sklearn.metrics import precision_recall_curve,accuracy_score,f1_score,precision_score,recall_score

import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import os

#from tensorboardX import SummaryWriter
from tqdm import tqdm

#4
import cv2
import torchvision.datasets as datasets
import torch.nn.functional as F
from tqdm import tqdm

#5
import torch.utils.data as data
import torch.nn.functional as F

#66
from easydict import EasyDict as edict
from pathlib import Path
from torch.nn import CrossEntropyLoss
from torchvision import transforms as trans
from torch.utils.data import Dataset, ConcatDataset, DataLoader
from torchvision import transforms as trans
from torchvision.datasets import ImageFolder
from PIL import Image, ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
import numpy as np
import cv2
import pickle


import splitfolders
from torch import nn, optim, as_tensor
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
from torch.optim import lr_scheduler
from torch.nn.init import *
from torchvision import transforms, utils, datasets, models
import cv2
from PIL import Image
from pdb import set_trace
import time
import copy
from pathlib import Path
import os
import sys
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from skimage import io, transform
from tqdm import trange, tqdm
import csv
import glob
import dlib
import pandas as pd
import numpy as np
from prettytable import PrettyTable
from google.colab import drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

ValueError: ignored

In [ ]:
%cd '/content/drive/MyDrive/Lowkey code/supp_material'

In [ ]:
%ls

In [ ]:
#convert to tensor 
to_tensor = transforms.ToTensor()

A little info about folders
1. Align,util, models and backbone are imported from lowkey. Align is a copy of similar folder from FaceEvolve.
2. Face is a clone of complete repository of FaceEvolve.

In [ ]:
import os
import sys
import math
import numbers

In [ ]:
from util.prepare_utils import prepare_models,prepare_dir_vec,get_ensemble,extract_features # Used to load attack backbone in code
from util.feature_extraction_utils import normalize_transforms,feature_extractor
from util.attack_utils import Attack
from align.detector import detect_faces
from align.align_trans import get_reference_facial_points,warp_and_crop_face
from backbone.model_irse import IR_152
import lpips
from face.applications.align.visualization_utils import show_results

#1

from face.head.metrics import ArcFace,CosFace,SphereFace

#2
from backbone.model_irse import IR_50, IR_101, IR_152, IR_SE_50, IR_SE_101, IR_SE_152
from backbone.model_resnet import ResNet_50, ResNet_101, ResNet_152

from face.loss.focal import FocalLoss

In [ ]:
if torch.cuda.is_available():
  device = torch.device("cuda")
else:
  device = torch.device("cpu")

In [ ]:
device

 Parameters for lowkey adversial Attack

In [ ]:
# Parameters for lowkey adversial Attack
eps = 0.05
n_iters = 50 # No of epooch for the attack
input_size = [112, 112] # Input size model expects 
attack_type = 'lpips' # attack type used
c_tv = None
c_sim = 0.05
lr = 0.0025 # the learning rate to optimize attack on
net_type = 'alex'
noise_size = 0.005 # noise size used
n_starts = 1
kernel_size_gf = 7 # gaussian smoothing window
sigma_gf = 3 # sigma for gaussain smoothing
combination = True # A switch to use gaussain smoothing together in attack or not
using_subspace = False
V_reduction_root = './'
direction = 1
crop_size = 112
scale = crop_size / 112 #1 


Parameters for training by Face Evolve. 

In [ ]:

EMBEDDING_SIZE = 512 # feature dimension
BATCH_SIZE = 512
DROP_LAST = True # whether drop the last batch to ensure consistent batch_norm statistics
LR = 0.1 # initial LR
NUM_EPOCH = 120 # total epoch number (use the firt 1/25 epochs to warm up)
WEIGHT_DECAY = 5e-4 # do not apply to batch_norm parameters
MOMENTUM = 0.9
STAGES = [35, 65, 95] # epoch stages to decay learning rate
HEAD_NAME = 'ArcFace' # support:  ['Softmax', 'ArcFace', 'CosFace', 'SphereFace', 'Am_softmax']
LOSS_NAME = 'Focal' # support: ['Focal', 'Softmax']

#INPUT_SIZE = [112, 112] # support: [112, 112] and [224, 224]
RGB_MEAN = [0.5, 0.5, 0.5] # for normalize inputs to [-1, 1]
       
#DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
MULTI_GPU = True # flag to use multiple GPUs; if you choose to train with single GPU, you should first run "export CUDA_VISILE_DEVICES=device_id" to specify the GPU card you want to use
GPU_ID = [0] # specify your GPU ids
PIN_MEMORY = True
NUM_WORKERS = 0

RGB_MEAN = [0.5, 0.5, 0.5] # for normalize inputs to [-1, 1]
RGB_STD = [0.5, 0.5, 0.5]

### 1. Load all the **Models**

---



In [ ]:
model_backbones = ['IR_152', 'IR_152', 'ResNet_152', 'ResNet_152'] # Model backbone for ensemble
model_roots = ['models/Backbone_IR_152_Arcface_Epoch_112.pth', 'models/Backbone_IR_152_Cosface_Epoch_70.pth', \
'models/Backbone_ResNet_152_Arcface_Epoch_65.pth', 'models/Backbone_ResNet_152_Cosface_Epoch_68.pth'] # Roots of the models 


In [ ]:
# Load an IR_152 Cosface model
PATH = 'models/Backbone_IR_152_Cosface_Epoch_70.pth'
IR_152C = IR_152(input_size=input_size)
IR_152C.load_state_dict(torch.load(PATH))

# Load IR_152 Arc Face model
PATH1 = 'models/Backbone_IR_152_Arcface_Epoch_112.pth'
IR_152A = IR_152(input_size=input_size)
IR_152A.load_state_dict(torch.load(PATH1))

#Load ResNet 152 Arc Face Model
PATH2 = 'models/Backbone_ResNet_152_Arcface_Epoch_65.pth'
RESNET_152A = ResNet_152(input_size=input_size)
RESNET_152A.load_state_dict(torch.load(PATH2))


# Load Resnet 152 CosFace Model
PATH3 = 'models/Backbone_ResNet_152_Cosface_Epoch_68.pth'
RESNET_152C = ResNet_152(input_size=input_size)
RESNET_152C.load_state_dict(torch.load(PATH3))



<All keys matched successfully>

In [ ]:
# Generate lowkey ensemble attack model
models_attack, V_reduction, dim = prepare_models(model_backbones,
              input_size,
              model_roots,
              kernel_size_gf,
              sigma_gf,
              combination,
              using_subspace,
              V_reduction_root)

Loading Attack Backbone Checkpoint '['models/Backbone_IR_152_Arcface_Epoch_112.pth', 'models/Backbone_IR_152_Cosface_Epoch_70.pth', 'models/Backbone_ResNet_152_Arcface_Epoch_65.pth', 'models/Backbone_ResNet_152_Cosface_Epoch_68.pth']'


##2. Split the whole face-scrub dataset into train and val 
 then move them to original folders. Make gallery =50 with 20 identities protected by lowkey. 
 

### 2. Now lets perform attacks on 5 identities
1. Ben Affleck 
2. Adam Brody 
3. Alfred Molina
4. Andy Garcia
5. Andy Serkis

---




In [ ]:
# Function to protect a single image

def protect_image(image):
  #img = Image.fromarray(img)
  img = Image.open(image)
  #print(img)
  reference = get_reference_facial_points(default_square=True) * scale
  print("The value for reference in get reference facial points  is {}".format(reference))
  h,w,c = np.array(img).shape
  print (h,w,c)

  b_box,landmarks = detect_faces(img) 
  #show_results(img, _, landmarks)
  print("The size of landmarks is {}".format(landmarks.size))  
  #print("The value for _ in detect faces is {}".format(_))
  # (landmarks)
  facial5points = [[landmarks[0][j], landmarks[0][j + 5]] for j in range(5)]
  #print("The value for _ in detect faces is {} end ".format(facial5points))
  b_box, tfm = warp_and_crop_face(np.array(img), facial5points, reference, crop_size=(crop_size, crop_size))
  print("The value for tfm in warp_and_crop_face is {}  ".format(tfm))
  # pytorch transform
  theta = normalize_transforms(tfm, w, h)
  tensor_img = to_tensor(img).unsqueeze(0).to(device)
  print("The value for theta in normalize_transforms(tfm, w, h) is {}  ".format(theta))
  V_reduction = None
  dim = 512

  # Find gradient direction vector
  dir_vec_extractor = get_ensemble(models = models_attack, sigma_gf=None, kernel_size_gf=None, combination=False, V_reduction=V_reduction, warp=True, theta_warp=theta)
  dir_vec = prepare_dir_vec(dir_vec_extractor, tensor_img, dim, combination)
  print("The value for direction vector of gradient in image is {}  ".format(dir_vec))
  img_attacked = tensor_img.clone()
  attack = Attack(models_attack, dim, attack_type, 
                  eps, c_sim, net_type, lr, n_iters, 
                  noise_size, n_starts, c_tv, sigma_gf, 
                  kernel_size_gf, combination, warp=True, 
                  theta_warp=theta, V_reduction=V_reduction)
  img_attacked = attack.execute(tensor_img, dir_vec, direction).detach().cpu()

  img_attacked_pil = transforms.ToPILImage()(img_attacked[0])
  return img_attacked_pil,dir_vec_extractor


In [ ]:
#Now lets create a function to attack a directory of images  with lowkey attack

def protect_dir(dirs_root,model):
  
  for img_name in tqdm(os.listdir(dirs_root)):

    
    img_root = os.path.join(dirs_root, img_name)
    #new_img_root = os.path.join(save_dir, img_name)
    print("The image processed is : {}".format(img_root))
    print('Finding reference points')
    reference = get_reference_facial_points(default_square=True) * scale
    img = Image.open(img_root)
    #h,w,c = np.array(img).shape
    h,w,c = np.array(img).shape
    #print("The value for reference in get reference facial points  is {}".format(reference))
    #print (h,w,c)
    ## Detects facial points ##
    b_box,landmarks = detect_faces(img) 
    facial5points = [[landmarks[0][j], landmarks[0][j + 5]] for j in range(5)]
    #print("The value for _ in detect faces is {} end ".format(facial5points))
    b_box, tfm = warp_and_crop_face(np.array(img), facial5points, reference, crop_size=(crop_size, crop_size))
    #print("The value for tfm in warp_and_crop_face is {}  ".format(tfm))
    
    ## find pytorch transform
    theta = normalize_transforms(tfm, w, h)
    tensor_img = to_tensor(img).unsqueeze(0).to(device)
    #print("The value for theta in normalize_transforms(tfm, w, h) is {}  ".format(theta))
    
    V_reduction = None
    dim = 512

  # Find gradient direction vector i.e find the feature vector extractor
    dir_vec_extractor = get_ensemble(models=model, sigma_gf=None, kernel_size_gf=None, combination=False, V_reduction=V_reduction, warp=True, theta_warp=theta)
    dir_vec = prepare_dir_vec(dir_vec_extractor, tensor_img, dim, combination).to(device)
  #print("The value for direction vector of gradient in image is {}  ".format(dir_vec))
  
  # Perform Low key attack
    img_attacked = tensor_img.clone()
    attack = Attack(model, dim, attack_type, 
                  eps, c_sim, net_type, lr, n_iters, 
                  noise_size, n_starts, c_tv, sigma_gf, 
                  kernel_size_gf, combination, warp=True, 
                  theta_warp=theta, V_reduction=V_reduction).to(device)
    img_attacked = attack.execute(tensor_img, dir_vec, direction).detach().cpu()
    print("attack Executed")
    img_attacked_pil = transforms.ToPILImage()(img_attacked[0])
    img_attacked_pil.save(img_root[:-4] + '.png')
    os.remove(img_root)


### 3.Lowkey ensemble attack on the 5 identities

In [ ]:
# protect_dir(dirs_root=lowkey_id_3,model=models_attack)


  0%|          | 0/58 [00:00<?, ?it/s]

The image processed is : /content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_galleries/adversial_gallery_ensemble/Andy_Serkis/Andy_Serkis_5918_3439.jpeg
Finding reference points
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /usr/local/lib/python3.8/dist-packages/lpips/weights/v0.1/alex.pth


  2%|▏         | 1/58 [00:36<34:26, 36.26s/it]

attack Executed
The image processed is : /content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_galleries/adversial_gallery_ensemble/Andy_Serkis/Andy_Serkis_5896_3424.jpeg
Finding reference points
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /usr/local/lib/python3.8/dist-packages/lpips/weights/v0.1/alex.pth


  3%|▎         | 2/58 [01:13<34:32, 37.00s/it]

...

attack Executed
The image processed is : /content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_galleries/adversial_gallery_ensemble/Andy_Serkis/Andy_Serkis_5904_3430.protected.png
Finding reference points
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /usr/local/lib/python3.8/dist-packages/lpips/weights/v0.1/alex.pth


 98%|█████████▊| 57/58 [35:15<00:36, 36.47s/it]

attack Executed
The image processed is : /content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_galleries/adversial_gallery_ensemble/Andy_Serkis/Andy_Serkis_5911_3435.protected.png
Finding reference points
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /usr/local/lib/python3.8/dist-packages/lpips/weights/v0.1/alex.pth


100%|██████████| 58/58 [35:51<00:00, 37.10s/it]

attack Executed


In [ ]:
# protect_dir(dirs_root=lowkey_id_4,model=models_attack)


  0%|          | 0/27 [00:00<?, ?it/s]

The image processed is : /content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_galleries/adversial_gallery_ensemble/Ben_Affleck/Ben_Affleck_8549_4727.jpeg
Finding reference points


/content/drive/MyDrive/Lowkey code/supp_material/align/first_stage.py:32: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  img = Variable(torch.FloatTensor(_preprocess(img)), volatile = True)
/content/drive/MyDrive/Lowkey code/supp_material/align/get_nets.py:70: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  a = F.softmax(a)
/content/drive/MyDrive/Lowkey code/supp_material/align/detector.py:81: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  img_boxes = Variable(torch.FloatTensor(img_boxes), volatile = True)
/content/drive/MyDrive/Lowkey code/supp_material/align/get_nets.py:115: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  a = F.softmax(a)
/content/drive/MyDrive/Lowkey code/supp_material/align/detector.py:102: UserWarning: volatile was remov

Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]


Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to /root/.cache/torch/hub/checkpoints/alexnet-owt-7be5be79.pth


...

attack Executed
The image processed is : /content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_galleries/adversial_gallery_ensemble/Ben_Affleck/Ben_Affleck_8461_4662.jpeg
Finding reference points
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /usr/local/lib/python3.8/dist-packages/lpips/weights/v0.1/alex.pth


 96%|█████████▋| 26/27 [16:03<00:36, 36.09s/it]

attack Executed
The image processed is : /content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_galleries/adversial_gallery_ensemble/Ben_Affleck/Ben_Affleck_8476_4672.jpeg
Finding reference points
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /usr/local/lib/python3.8/dist-packages/lpips/weights/v0.1/alex.pth


100%|██████████| 27/27 [16:38<00:00, 37.00s/it]

attack Executed


In [ ]:
protect_dir(dirs_root=lowkey_id_5,model=models_attack)


  0%|          | 0/66 [00:00<?, ?it/s]

The image processed is : /content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/Ensemble_attack/gallery/4.Amaury_Nolasco/Amaury_Nolasco_4617_2585.jpeg
Finding reference points


/content/drive/MyDrive/Lowkey code/supp_material/align/first_stage.py:32: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  img = Variable(torch.FloatTensor(_preprocess(img)), volatile = True)
/content/drive/MyDrive/Lowkey code/supp_material/align/get_nets.py:70: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  a = F.softmax(a)
/content/drive/MyDrive/Lowkey code/supp_material/align/detector.py:81: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  img_boxes = Variable(torch.FloatTensor(img_boxes), volatile = True)
/content/drive/MyDrive/Lowkey code/supp_material/align/get_nets.py:115: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  a = F.softmax(a)
/content/drive/MyDrive/Lowkey code/supp_material/align/detector.py:102: UserWarning: volatile was remov

Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]


Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to /root/.cache/torch/hub/checkpoints/alexnet-owt-7be5be79.pth


...

attack Executed
The image processed is : /content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/Ensemble_attack/gallery/4.Amaury_Nolasco/Amaury_Nolasco_4914_2767.jpeg
Finding reference points
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /usr/local/lib/python3.8/dist-packages/lpips/weights/v0.1/alex.pth


 98%|█████████▊| 65/66 [38:31<00:34, 34.90s/it]

attack Executed
The image processed is : /content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/Ensemble_attack/gallery/4.Amaury_Nolasco/Amaury_Nolasco_4956_2788.jpeg
Finding reference points
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /usr/local/lib/python3.8/dist-packages/lpips/weights/v0.1/alex.pth


100%|██████████| 66/66 [39:07<00:00, 35.57s/it]

attack Executed
